In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install pytorch-lightning

In [ ]:
import pytorch_lightning as pl
import pandas as pd
import cv2
import os 
from torch import nn
from torch.utils.data import Dataset ,DataLoader, random_split
import numpy as np
import torch
from sklearn.model_selection import train_test_split 
from torchvision import transforms, datasets, models
import matplotlib.pyplot as plt
import torchmetrics
from torchmetrics.functional import accuracy
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torchvision.utils import make_grid
import math
import torch.nn.functional as F

In [ ]:
pl.seed_everything(7)
PATH_DATASETS = os.environ.get("PATH_DATASETS", "/content/drive/MyDrive/Datasets")
BATCH_SIZE = 64 
CLASS_SIZE = 10
VAL_SIZE = 10000

Global seed set to 7


In [ ]:
class Data(pl.LightningDataModule):
  def __init__(self, data_dir: str=PATH_DATASETS, batch_size: int=BATCH_SIZE, class_size: int=CLASS_SIZE,
               val_size: int=VAL_SIZE):
    super().__init__()
    self.data_dir = data_dir
    self.batch_size = batch_size
    self.class_size = class_size
    self.val_size = val_size
    self.transform = {
        'train': transforms.Compose([
            transforms.Resize((224,224)),
            transforms.RandomHorizontalFlip(p=0.7),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ]),

        'test': transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
    }

    

  def prepare_data(self):
    datasets.CIFAR10(root=self.data_dir, train=True, download=True)
    datasets.CIFAR10(root=self.data_dir, train=False, download=True)

  def setup(self, stage=None):
    if stage == 'fit' or stage is None:
      cifar10_full = datasets.CIFAR10(root=self.data_dir, train=True, transform=self.transform['train'])
      self.train_size = len(cifar10_full) - self.val_size
      self.cifar_train, self.cifar_val = random_split(cifar10_full, [self.train_size, self.val_size])

    if stage == 'test' or stage is None:
      self.cifar_test = datasets.CIFAR10(root=self.data_dir, train=False, transform=self.transform['test'])

  def train_dataloader(self):
    return DataLoader(self.cifar_train, batch_size=32, shuffle=True)

  def val_dataloader(self):
    return DataLoader(self.cifar_val, batch_size=32, shuffle=False)

  def test_dataloader(self):
    return DataLoader(self.cifar_test, batch_size=32, shuffle=False)

  def visualize(self):
    for images, _ in self.train_dataloader():
      print(f"images.shape: {images.shape}")
      plt.figure(figsize=(16,8))
      plt.axis("off")
      plt.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
      break

In [ ]:
dm = Data()
dm.prepare_data()

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
dm.setup()

In [ ]:
type(dm.cifar_train)

torch.utils.data.dataset.Subset

In [ ]:
class LitResnet50(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = models.resnet50()

  def forward(self, x):
    x = self.model(x)
    return x

  def training_step(self, batch, batch_idx):
    x, y = batch
    logits = self.forward(x)
    loss = F.cross_entropy(logits, y)
    self.log("train loss", loss)
    return loss

  def evaluate(self, batch, stage=None):
    x, y = batch
    logits = self.forward(x)
    loss = F.cross_entropy(logits, y)
    preds = torch.argmax(logits, dim=1)
    acc = accuracy(preds, y)
    
    if stage:
      self.log(f'{stage}_loss', loss, prog_bar=True)
      self.log(f'{stage}_acc', acc, prog_bar=True)

  def validation_step(self, batch, batch_idx):
    return self.evaluate(batch, stage="val")

  def test_step(self, batch, batch_idx):
    return self.evaluate(batch, stage="test")

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=1e-4)

In [ ]:
model = LitResnet50()

trainer = pl.Trainer(max_epochs=1,
                  accelerator="auto",
                  devices=1 if torch.cuda.is_available() else None,
                  callbacks=[LearningRateMonitor(logging_interval="step"),
                                TQDMProgressBar(refresh_rate=20)],
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, dm)

Files already downloaded and verified
Files already downloaded and verified


Missing logger folder: /content/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 25.6 M
---------------------------------
25.6 M    Trainable params
0         Non-trainable params
25.6 M    Total params
102.228   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
trainer.validate(model, dm)

Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc            0.47850000858306885
        val_loss            1.4335651397705078
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_acc': 0.47850000858306885, 'val_loss': 1.4335651397705078}]

In [ ]:
trainer.test(model, dm)

Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.48339998722076416
        test_loss           1.4066582918167114
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.48339998722076416, 'test_loss': 1.4066582918167114}]